## Uvod v regresijo - Lekcija 1

#### Postavljanje v perspektivo

✅ Obstaja veliko vrst regresijskih metod, izbira pa je odvisna od odgovora, ki ga iščete. Če želite napovedati verjetno višino osebe glede na njeno starost, bi uporabili `linearno regresijo`, saj iščete **številčno vrednost**. Če vas zanima, ali naj se določena vrsta kuhinje šteje za vegansko ali ne, iščete **dodelitev kategorije**, zato bi uporabili `logistično regresijo`. Več o logistični regresiji boste izvedeli kasneje. Razmislite o nekaterih vprašanjih, ki jih lahko zastavite podatkom, in o tem, katera od teh metod bi bila bolj primerna.

V tem razdelku boste delali z [majhnim naborom podatkov o diabetesu](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Predstavljajte si, da želite preizkusiti zdravljenje za diabetične bolnike. Modeli strojnega učenja vam lahko pomagajo določiti, kateri bolniki bi se na zdravljenje bolje odzvali, glede na kombinacije spremenljivk. Tudi zelo osnovni regresijski model, ko je vizualiziran, lahko pokaže informacije o spremenljivkah, ki bi vam pomagale organizirati teoretične klinične preizkuse.

Pa začnimo s to nalogo!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Umetniško delo @allison_horst</figcaption>

<!--![Umetniško delo \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Umetniško delo @allison_horst-->


## 1. Nalaganje našega nabora orodij

Za to nalogo bomo potrebovali naslednje pakete:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka paketov za R](https://www.tidyverse.org/packages), zasnovana za hitrejše, enostavnejše in bolj zabavno podatkovno znanost!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je [zbirka paketov](https://www.tidymodels.org/packages/) za modeliranje in strojno učenje.

Namestite jih lahko z ukazom:

`install.packages(c("tidyverse", "tidymodels"))`

Spodnji skript preveri, ali imate nameščene pakete, potrebne za dokončanje tega modula, in jih po potrebi namesti.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Zdaj naložimo te odlične pakete in jih naredimo dostopne v naši trenutni R seji. (To je zgolj za ponazoritev, `pacman::p_load()` je to že naredil namesto vas)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Dataset o diabetesu

V tej vaji bomo uporabili svoje regresijske veščine za napovedovanje na podlagi podatkov o diabetesu. [Dataset o diabetesu](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) vključuje `442 vzorce` podatkov o diabetesu, z 10 spremenljivkami napovedovalnih značilnosti: `starost`, `spol`, `indeks telesne mase`, `povprečni krvni tlak` in `šest meritev krvnega seruma`, ter izhodno spremenljivko `y`: kvantitativno merilo napredovanja bolezni eno leto po začetnem stanju.

|Število opazovanj|442|
|-----------------|:---|
|Število napovedovalcev|Prvih 10 stolpcev je numeričnih napovedovalnih|
|Izhodna tarča|Stolpec 11 je kvantitativno merilo napredovanja bolezni eno leto po začetnem stanju|
|Informacije o napovedovalcih|- starost v letih
||- spol
||- bmi indeks telesne mase
||- bp povprečni krvni tlak
||- s1 tc, skupni serumski holesterol
||- s2 ldl, lipoproteini nizke gostote
||- s3 hdl, lipoproteini visoke gostote
||- s4 tch, skupni holesterol / HDL
||- s5 ltg, verjetno logaritem ravni serumskih trigliceridov
||- s6 glu, raven krvnega sladkorja|

> 🎓 Zapomnite si, to je nadzorovano učenje, zato potrebujemo ciljno spremenljivko 'y'.

Preden lahko manipulirate s podatki v R-ju, jih morate uvoziti v pomnilnik R-ja ali vzpostaviti povezavo do podatkov, ki jih R lahko uporablja za dostop do podatkov na daljavo.

> Paket [readr](https://readr.tidyverse.org/), ki je del Tidyverse, ponuja hiter in prijazen način za branje pravokotnih podatkov v R.

Zdaj naložimo dataset o diabetesu, ki je na voljo na tem URL-ju: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Prav tako bomo izvedli osnovni pregled podatkov z uporabo `glimpse()` in prikazali prvih 5 vrstic z uporabo `slice()`.

Preden nadaljujemo, naj predstavimo nekaj, kar boste pogosto srečali v kodi R 🥁🥁: operator cevi `%>%`

Operator cevi (`%>%`) izvaja operacije v logičnem zaporedju tako, da objekt posreduje naprej v funkcijo ali izraz klica. Operator cevi si lahko predstavljate kot "in nato" v vaši kodi.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` nam pokaže, da ima ta podatkovni niz 442 vrstic in 11 stolpcev, pri čemer so vsi stolpci tipa podatkov `double`.

<br>

> `glimpse()` in `slice()` sta funkciji v knjižnici [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, ki je del Tidyverse, je slovnica za manipulacijo podatkov, ki ponuja dosleden nabor glagolov za reševanje najpogostejših izzivov pri obdelavi podatkov.

<br>

Zdaj, ko imamo podatke, se osredotočimo na eno značilnost (`bmi`), ki jo bomo uporabili za to vajo. To zahteva, da izberemo želene stolpce. Kako to storimo?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) nam omogoča, da *izberemo* (in po želji preimenujemo) stolpce v podatkovnem okviru.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Podatki za učenje in testiranje

Pri nadzorovanem učenju je običajna praksa, da podatke *razdelimo* na dva podsklopa; (običajno večji) sklop, s katerim treniramo model, in manjši "rezervni" sklop, s katerim preverimo, kako se je model odrezal.

Zdaj, ko imamo podatke pripravljene, lahko preverimo, ali nam stroj lahko pomaga določiti logično razdelitev med številkami v tem naboru podatkov. Uporabimo lahko paket [rsample](https://tidymodels.github.io/rsample/), ki je del okvira Tidymodels, za ustvarjanje objekta, ki vsebuje informacije o *načinu* razdelitve podatkov, nato pa še dve funkciji rsample za pridobitev ustvarjenih učnih in testnih sklopov:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Učite model linearne regresije s Tidymodels

Zdaj smo pripravljeni, da naučimo naš model!

V Tidymodels modele določite z uporabo `parsnip()` tako, da določite tri koncepte:

-   **Tip** modela razlikuje med modeli, kot so linearna regresija, logistična regresija, modeli odločitvenih dreves in podobno.

-   **Način** modela vključuje pogoste možnosti, kot sta regresija in klasifikacija; nekateri tipi modelov podpirajo oba načina, medtem ko imajo drugi le en način.

-   **Pogon** modela je računsko orodje, ki bo uporabljeno za prilagoditev modela. Pogosto so to R paketi, kot sta **`"lm"`** ali **`"ranger"`**.

Te informacije o modeliranju so zajete v specifikaciji modela, zato jo ustvarimo!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Ko je model *določen*, ga je mogoče `oceniti` ali `usposobiti` z uporabo funkcije [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), običajno z uporabo formule in nekaterih podatkov.

`y ~ .` pomeni, da bomo prilagodili `y` kot napovedano količino/cilj, ki ga pojasnjujejo vsi napovedniki/lastnosti, tj. `.` (v tem primeru imamo samo en napovednik: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Iz modelnega izhoda lahko vidimo koeficiente, pridobljene med učenjem. Ti predstavljajo koeficiente premice najboljše prileganja, ki nam daje najnižjo skupno napako med dejansko in napovedano spremenljivko.
<br>

## 5. Napovedovanje na testnem naboru

Zdaj, ko smo izurili model, ga lahko uporabimo za napovedovanje napredovanja bolezni y za testni nabor podatkov z uporabo [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). To bo uporabljeno za risanje premice med skupinami podatkov.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Juhu! 💃🕺 Pravkar smo izurili model in ga uporabili za napovedovanje!

Pri napovedovanju je konvencija tidymodels vedno ustvariti tibble/podatkovni okvir z rezultati in standardiziranimi imeni stolpcev. To omogoča enostavno združevanje izvirnih podatkov in napovedi v uporabni obliki za nadaljnje operacije, kot je risanje grafov.

`dplyr::bind_cols()` učinkovito združi več podatkovnih okvirjev po stolpcih.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Prikaz rezultatov modeliranja

Zdaj je čas, da to vidimo vizualno 📈. Ustvarili bomo razpršen diagram vseh vrednosti `y` in `bmi` iz testnega nabora, nato pa uporabili napovedi za risanje črte na najbolj ustreznem mestu med skupinami podatkov modela.

R ima več sistemov za izdelavo grafov, vendar je `ggplot2` eden najbolj elegantnih in najbolj vsestranskih. Omogoča vam sestavljanje grafov z **združevanjem neodvisnih komponent**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Malo razmislite, kaj se tukaj dogaja. Ravna črta poteka skozi številne majhne točke podatkov, vendar kaj točno počne? Ali vidite, kako bi morali biti sposobni uporabiti to črto za napovedovanje, kje bi se nova, nevidena podatkovna točka morala uvrstiti glede na y-os grafa? Poskusite z besedami opisati praktično uporabo tega modela.

Čestitke, zgradili ste svoj prvi model linearne regresije, z njim ustvarili napoved in jo prikazali na grafu!



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
